In [30]:
import pandas as pd
#import matplotlib.ticker as mtick
import numpy as np
import arcpy
from arcgis.gis import *
import datetime
gis = GIS()

## Check Subareas

In [10]:
# Here, we visually inspect if all the shapefiles meet the following criteria
## 1. The shapefile only includes those segments whose centroid falls within the corresponding subarea
## 2. A SUBAREAID column exists

# The IronCo Segment file does not include a SUBAREAID column, so we add one: 
input_shapefile = r'../data/5_IronCo - v1.0 - 2023-09-13_DRAFT/Segments_IR_20230912b/Segments_IR_20230912b.shp'
output_shapefile = input_shapefile.replace('b.shp', 'c.shp')

arcpy.management.CopyFeatures(input_shapefile, output_shapefile)
arcpy.management.AddField(output_shapefile, "SUBAREAID", "SHORT")
arcpy.management.CalculateField(output_shapefile, "SUBAREAID", 5)

<Result '../data/5_IronCo - v1.0 - 2023-09-13_DRAFT/Segments_IR_20230912b/Segments_IR_20230912c.shp'>

## Set Filepaths

In [31]:
# Set the workspace environment to the folder where the output shapefile will be stored
arcpy.env.workspace = 'D:/GitHub/UDOT-Master-Segments/outputs/'

# Output merged shapefile path
joinShp = 'Merged_Segments.shp'

# List of input shapefile paths
shpPaths = [
    '../data/0_USTM_v3.0 - 2023-08-17_DRAFT/Segments_UD_20230729/Segments_UD_20220729b.shp',
    '../data/1_WF/Segments_WF_20230919/Segments_WF_20230919.shp',
    '../data/2_Cache/Segments_CA_20230302/Segments_CA_20230302.shp',
    '../data/3_Dixie/Segments_DX_20220915b/Segments_DX_20220915b.shp',
    '../data/4_SuWsv2_2023-09-13_DRAFT/Segments_SW_20230913/Segments_SW_20230913.shp',
    '../data/5_IronCo - v1.0 - 2023-09-13_DRAFT/Segments_IR_20230912b/Segments_IR_20230912c.shp'
]

## Merge Segments

In [37]:
# Use arcpy.management.Merge with FieldMappings to merge the shapefiles
fieldMappings = arcpy.FieldMappings()
fieldMappings.mergeRule = 'Join'
arcpy.management.Merge(shpPaths, joinShp, fieldMappings)

<Result 'D:/GitHub/UDOT-Master-Segments/outputs\\Merged_Segments.shp'>

## Check CRS

In [13]:
try:
    # Use arcpy.Describe to get information about the shapefile
    desc = arcpy.Describe(joinShp)
    
    # Check if the shapefile has a spatial reference (projection)
    if desc.spatialReference is not None:
        # Get the name of the coordinate system
        coordinate_system_name = desc.spatialReference.name
        
        # Print the coordinate system information
        print(f"Coordinate System: {coordinate_system_name}")
    else:
        print("The shapefile does not have a defined coordinate system.")
except arcpy.ExecuteError:
    print(arcpy.GetMessages(2))
except Exception as e:
    print(str(e))

Coordinate System: NAD_1983_UTM_Zone_12N


## Check for Duplicates

In [14]:
# Create a set to store unique SEGID values
uniqueSegIDs = set()

# Create a list to store duplicate SEGID values
duplicateSegIDs = []

# Use a SearchCursor to iterate through the SEGID field
with arcpy.da.SearchCursor(joinShp, ['SEGID']) as cursor:
    for row in cursor:
        segid = row[0]
        if segid in uniqueSegIDs:
            duplicateSegIDs.append(segid)
        else:
            uniqueSegIDs.add(segid)

# Check if there are any duplicate SEGIDs
if len(duplicateSegIDs) > 0:
    print("Duplicate SEGIDs found in Merged_Segments.shp:")
    for segid in duplicateSegIDs:
        print(segid)
else:
    print("No duplicate SEGIDs found in Merged_Segments.shp.")

No duplicate SEGIDs found in Merged_Segments.shp.


## Selecting Necessary Fields

In [38]:
# Specify the input shapefile path
fields_to_keep = ['FID', 'Shape', 'SEGID', 'BMP', 'EMP', 'DISTANCE', 'CO_FIPS', 'PLANAREA', 'SUBAREAID', 'FAC_WDAVG']
inputShp = r'D:/GitHub/UDOT-Master-Segments/outputs/Merged_Segments.shp'

try:
    # Delete unwanted fields from the copied shapefile
    fields_to_delete = [field.name for field in arcpy.ListFields(inputShp) if field.name not in fields_to_keep]
    arcpy.management.DeleteField(inputShp, fields_to_delete)

    print("Columns deleted successfully.")
except arcpy.ExecuteError:
    print(arcpy.GetMessages(2))
except Exception as e:
    print(str(e))

Columns deleted successfully.


## Recalculate the DISTANCE Field

In [40]:
# Specify the input shapefile path
inputShp = r'Merged_Segments.shp'

# Specify the name of the new field
distanceField = "DISTANCE"

# Specify the output shapefile path
outputShp = r'Final_Segments.shp'

# Conversion factor from meters to miles with 8 decimal precision
meters_to_miles = 0.000621371192

try:
      
    # Copy the input shapefile to create a new output shapefile
    arcpy.management.CopyFeatures(inputShp, outputShp)

    # Create an update cursor to calculate the DISTANCE2 field
    with arcpy.da.UpdateCursor(outputShp, ['SHAPE@', distanceField]) as cursor:
        for row in cursor:
            # Get the segment's geometry and calculate its length in meters
            segment_geometry = row[0]
            segment_length = segment_geometry.length * meters_to_miles

            # Update the DISTANCE2 field with the calculated length in meters
            row[1] = segment_length
            cursor.updateRow(row)
    
    # print statements for checking
    print(f"{newField} field calculated successfully.")
    df = pd.DataFrame.spatial.from_featureclass(outputShp)
    print(df[['DISTANCE']].head(5))
            
    # Delete Merged_Segments
    #arcpy.management.Delete(inputShp)
    
except arcpy.ExecuteError:
    print(arcpy.GetMessages(2))
except Exception as e:
    print(str(e))

DISTANCE2 field calculated successfully.
    DISTANCE
0   0.666642
1  15.369870
2  30.002021
3  14.194335
4  17.323272
